In [14]:
import re
import random

In [11]:
with open('dataset/data_ner.txt', 'r') as f:
    dataset = []
    for line in f:
        text = line.strip('\n')
        dataset.append(text)
    
first_name = []
with open('statics/person_names_female_th.txt', 'r') as f:
    first_name.extend([i.strip('\n') for i in f])
with open('statics/person_names_male_th.txt', 'r') as f:
    first_name.extend([i.strip('\n') for i in f])
    
with open('statics/family_names_th.txt', 'r') as f:
    family_name = [i.strip('\n') for i in f]

In [12]:
person_start = re.compile(r'\[PERSON\]')
person_end = re.compile(r'\[/PERSON\]')

In [15]:
existed_first_name = []
existed_family_name = []
dataset_new = []

for data in dataset:
    if('[PERSON]' in data):
        new_data = data
        exist_data = data
        for start, end in zip(person_start.finditer(data), person_end.finditer(data)):
            current_name = data[start.end():end.start()].split(' ')
            
            new_name = ''
            if(len(current_name) > 1):
                if(current_name[0] in existed_first_name or current_name[1] in existed_family_name):
                    random_first = random.choice(first_name)
                    random_last = random.choice(family_name)

                    existed_first_name.append(random_first)
                    existed_family_name.append(random_last)
                    first_name.remove(random_first)
                    family_name.remove(random_last)
                    
                    exist_data = exist_data.replace(' '.join(current_name), f'{random_first} {random_last}')
                else: 
                    existed_first_name.append(current_name[0])
                    existed_family_name.append(current_name[1])
                
                    if(current_name[0] in first_name):
                        first_name.remove(current_name[0])
                    elif(current_name[1] in family_name):
                        family_name.remove(current_name[1])
                
                random_first = random.choice(first_name)
                random_last = random.choice(family_name)
                
                existed_first_name.append(random_first)
                existed_family_name.append(random_last)
                first_name.remove(random_first)
                family_name.remove(random_last)
                new_name = f'{random_first} {random_last}'
            
            else:
                if(current_name[0] in existed_first_name):
                    random_first = random.choice(first_name)

                    existed_first_name.append(random_first)
                    first_name.remove(random_first)
                    
                    exist_data = exist_data.replace(' '.join(current_name), random_first)
                else:
                    existed_first_name.append(current_name[0])

                    if(current_name[0] in first_name):
                        first_name.remove(current_name[0])
                    
                random_first = random.choice(first_name)
                existed_first_name.append(random_first)
                first_name.remove(random_first)
                new_name = random_first
            
            new_data = new_data.replace(' '.join(current_name), new_name)
            
        if(exist_data != data):
            dataset_new.append(exist_data)
        else:
            dataset_new.append(data)
        dataset_new.append(new_data)
    else:
        dataset_new.append(data)

In [18]:
with open('dataset/generated_ner.txt', 'w') as f:
    f.write('\n'.join(dataset_new))